In [2]:
import pandas as pd
import os
from omegaconf import DictConfig 
from inference.read_report import ReportProcessor
from inference.compute_post_scores import ComputePostScore
import hydra
from hydra.utils import instantiate
from inference.visualization import DataVisualization
from inference.performance_statistics import *

In [2]:
def run_computation(cfg:DictConfig):
    processor = ReportProcessor(cfg)
    result_df = processor.read_metric_reports()
    InD_scenarios= load_scenario_types_from_csv('scenario_type_counts.csv')
    labeled_df = label_scenarios(result_df, InD_scenarios) 
    compute_postscore=ComputePostScore(labeled_df, cfg)
    energy_score = compute_postscore.get_min_score()
    average_energy_score = compute_postscore.calculate_average_ood_score()
    visualizer = DataVisualization(figsize=(12, 8), alpha=0.6, grid=True)
    visualizer.draw_distribution(average_energy_score, score='ood_score_avg')

In [58]:
CONFIG_PATH = 'inference/config'
CONFIG_NAME = 'runner_report'
hydra.core.global_hydra.GlobalHydra.instance().clear()
hydra.initialize(config_path=CONFIG_PATH)
# cfg = hydra.compose(config_name=CONFIG_NAME,overrides=['split=test','planner=planTF', 'job_name=open_loop_boxes'])
cfg = hydra.compose(config_name=CONFIG_NAME,overrides=['job_name=open_loop_boxes'])
# cfg = hydra.compose(config_name=CONFIG_NAME)
# cfg = hydra.compose(config_name=CONFIG_NAME,overrides=['job_name=closed_loop_reactive_agents'])
processor = ReportProcessor(cfg)
result_df = processor.read_metric_reports()

In [63]:
result_df=calculate_average_metric_score(result_df)
result_df=label_low_score(result_df)

In [59]:
result_df = add_risk_label(result_df)

ValueError: The following metrics are missing from the DataFrame: ['no_ego_at_fault_collisions', 'drivable_area_compliance', 'driving_direction_compliance', 'time_to_collision_within_bound', 'ego_progress_along_expert_route', 'ego_is_making_progress', 'ego_is_comfortable', 'speed_limit_compliance']